In [3]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pickle
from torch import 
torch.manual_seed(1)

In [4]:
#try to load the entire data set
#load data
train_r = pickle.load(open("train_r.pkl", "rb"))
train_b = pickle.load(open("train_b.pkl", "rb"))
trainX = np.concatenate((train_r, train_b))

valid_r = pickle.load(open("test_r.pkl", "rb"))
valid_b = pickle.load(open("test_b.pkl", "rb"))
testX = np.concatenate((valid_r, valid_b))

#concatenate everythin appropriately to have the final datasets
one_label = np.ones(len(train_r))
zeros_label = np.zeros(len(train_b))
one_test = np.ones(len(valid_r))
zero_test = np.zeros(len(valid_b))

trainX = np.concatenate((train_r, train_b))
trainY = np.concatenate((one_label, zeros_label))
validX = np.concatenate((valid_r, valid_b))
validY  = np.concatenate((one_test, zero_test))

In [5]:
#gerenate feature vectors 
total_words = {}
for line in trainX:
    words = line.split(" ")
    for word in words:
        if word == "":
            continue
        try:
            total_words[word]
        except:
            total_words[word] = True


print(len(total_words.keys()))
#dont add words in valid and test sets

4818


In [6]:
unique_keys = list(total_words.keys())
unique_keys.sort()
#print(unique_keys[:10])
#account for a bias
train_feats = np.zeros((len(trainX), len(total_words.keys()) ))
for i, line in enumerate(trainX):
    words = line.split(" ")
    #print(words)
    for word in words:
        if word =="":
            continue
        #print(unique_keys.index(word))
        ind = unique_keys.index(word)  
        #plus one is so that the very first is the bias (here a bias of zero)
        train_feats[i][ind] = 1
        
valid_feats = np.zeros((len(validX), len(total_words.keys()) ))
for i, line in enumerate(validX):
    words = line.split(" ")
    #print(words)
    for word in words:
        if word =="":
            continue
        #account for value error:
        try:
            ind = unique_keys.index(word) 
            #print(ind)
            valid_feats[i][ind] = 1
        except:
            continue  
print(train_feats.shape, valid_feats.shape)

(2287, 4818) (491, 4818)


In [7]:
#create word to index embedding
word_to_ix = {}
for i in range(len(unique_keys)):
    word_to_ix[unique_keys[i]] = i
print(word_to_ix["trump"], word_to_ix["hillary"])
#word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(len(unique_keys), 10)  # 4818 words in vocab, 10 dimensional embeddings

lookup_tensor = torch.LongTensor([word_to_ix["trump"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)
lookup_tensor = torch.LongTensor([word_to_ix["donald"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

4413 2016
Variable containing:
-1.0018 -0.2858  1.3578  0.1852 -0.1547  1.4837  0.1961 -0.0178  0.0799 -1.0552
[torch.FloatTensor of size 1x10]

Variable containing:
-1.9980  0.6409 -1.7913 -0.5456  1.4099 -0.2040  0.9456 -0.0372  0.4437  0.4592
[torch.FloatTensor of size 1x10]



In [8]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        self.features = nn.Sequential(
        # an affine operation: y = Wx + b
        nn.Linear(len(unique_keys), 200),
        nn.ReLU(),
        #nn.Linear(2000, 200),
        #nn.ReLU(),
        #nn.Dropout( 0.1),
        nn.Linear(200, 20),
        nn.ReLU(),
        nn.Linear(20, 2), # final two classees, real or fake
    )

    def forward(self, x):
        # If the size is a square you can only specify a single number
        x = self.features(x)
        return x

net = Net()
print(net)


Net(
  (features): Sequential(
    (0): Linear(in_features=4818, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)


In [19]:
#train_feats, trainY
dtype_float = torch.FloatTensor
dtype_long = torch.LongTensor
print(train_feats.shape)#, valid_feats
loss_fn = torch.nn.CrossEntropyLoss()
from torch.autograd import Variable
x_data = Variable(torch.from_numpy(train_feats), requires_grad = False).type(dtype_float)
x_valid_data = Variable(torch.from_numpy(valid_feats), requires_grad = False).type(dtype_float)
y_classes = Variable(torch.from_numpy(trainY), requires_grad = False).type(dtype_long)

#net.forward(x_data)
print(y_classes.shape)
learning_rate = 5e-3
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate, weight_decay=0.01)
#softmax = torch.nn.Softmax()
net.forward(x_data)


(2287, 4818)
torch.Size([2287])


Variable containing:
-0.1632  0.1632
-0.1800  0.1800
-0.2545  0.2545
       ⋮        
-0.1225  0.1225
-0.1315  0.1315
-0.2171  0.2171
[torch.FloatTensor of size 2287x2]

In [23]:
%%time
for t in range(1000):
    #print 'hey'
    y_pred = net(x_data)
    #print y_pred
    loss = loss_fn(y_pred, y_classes)
    
    optimizer.zero_grad() #model.zero_grad()  # Zero out the previous gradient computation
    loss.backward()    # Compute the gradient
    optimizer.step()   # Use the gradient information to 
                   # make a step
    print(t, loss.data[0])
    #if t %100 ==99:
    #    print(t)

#print "hel

0 0.044257257133722305
1 0.04481954872608185
2 0.044575586915016174
3 0.044896744191646576
4 0.044582001864910126
5 0.04453561082482338
6 0.044598836451768875
7 0.044618237763643265
8 0.04470000043511391
9 0.04449779540300369
10 0.04489513859152794
11 0.04465881362557411
12 0.0452129952609539
13 0.04470394551753998
14 0.0455005019903183
15 0.04487275332212448
16 0.04564740136265755
17 0.044751282781362534
18 0.04531065747141838
19 0.04439763352274895
20 0.044974662363529205
21 0.0444319024682045
22 0.044920872896909714
23 0.0445597879588604
24 0.04481373354792595
25 0.0444490872323513
26 0.04486396536231041
27 0.0446259081363678
28 0.0451539047062397
29 0.04461017996072769
30 0.04531194269657135
31 0.04480822756886482
32 0.04561197757720947
33 0.04472830519080162
34 0.04534002020955086
35 0.04450760781764984
36 0.045104824006557465
37 0.04443218559026718
38 0.04490689933300018
39 0.044515758752822876
40 0.044864024966955185
41 0.0444936566054821
42 0.04479656741023064
43 0.044553376734

In [27]:
net.eval()
y_pred = net(x_valid_data)
preds = torch.max(y_pred, 1)
#print(preds[1].data)
from sklearn import metrics
#print(x_data.shape, x_valid_data.shape)
#print(preds)
y_train_pred = net(x_data)
train_preds = torch.max(y_train_pred, 1)

In [28]:
print(metrics.accuracy_score(trainY, train_preds[1].data))
print(metrics.average_precision_score(trainY, train_preds[1].data))

print(metrics.accuracy_score(validY, preds[1].data))
print(metrics.average_precision_score(validY, preds[1].data))

0.9995627459554001
0.9992748368382887
0.8431771894093686
0.8236384260315014
